## IBM Applied Data Science Capstone - Week 5 Final Project

### In this project we aim to find the best cluster to open Cafe bar in Hamilton, New zealnad. To do:

1. A dataframe is created to store neighborhoods in Hamilton New zeland by using anf scraping data from Wikipedia page.

2. Foursquare API is used to capture all the vanue data of the neighborhood. To do this at first geographical coordinates are generated.

3. Neighborhood is clustered in three clusters and finally the best cluster that has the highest potential to open a cafe bar is selected.


In [3]:
# library to handle requests
import requests 

# library for data analsysis
import pandas as pd

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# library to handle data in a vectorized manner
import numpy as np 

# library for random number generation
import random 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
!pip install geocoder

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import folium # plotting library
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
# send the GET request to wiki page to get data
data = requests.get('https://en.wikipedia.org/wiki/Category:Suburbs_of_Hamilton,_New_Zealand').text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    #print (row.text) # this can be use to make sure all data is captures from webpage
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
Neighborhood_df = pd.DataFrame({"Neighborhood": neighborhoodList})

Neighborhood_df.head()

,Neighborhood
0,"Suburbs of Hamilton, New Zealand"
1,"Ashmore, New Zealand"
2,Beerescourt
3,Callum Brae
4,"Chartwell, Hamilton"


In [9]:
# print the dimensions of the Neighborhood_df dataframe
Neighborhood_df.shape

(50, 1)

In [10]:
# define a function to get coordinates
# use while loop and loop till the last coordinate is captured 
def get_coords(neighborhood):
 
    coords = None

    while(coords is None):
        g = geocoder.arcgis('{}, Hamilton, New Zealand'.format(neighborhood))
        coords = g.latlng
    return coords

In [11]:
# call function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_coords(neighborhood) for neighborhood in Neighborhood_df["Neighborhood"].tolist() ]

In [12]:
coordinates

[[-37.82651999999996, 175.22323000000006],
 [-37.82651999999996, 175.22323000000006],
 [-37.76720324499996, 175.26269042],
 [-37.73730545520315, 175.2700519910966],
 [-37.737680212800264, 175.27459245951817],
 [-37.75324999999998, 175.2838700000001],
 [-37.77859999999998, 175.28703000000007],
 [-42.28723356399996, 172.34343250000006],
 [-37.826033423999945, 175.2762703830001],
 [-37.79547999999994, 175.24581000000012],
 [-37.76819334999993, 175.29682201800006],
 [-37.76933809248405, 175.27499062029574],
 [-37.75884858799998, 175.3189430440001],
 [-37.815270936439845, 175.2966677103643],
 [-37.726032283999984, 175.24516020900012],
 [-37.771927732999984, 175.2531433050001],
 [-37.788809999999955, 175.26127000000008],
 [-37.82242999999994, 175.28757000000007],
 [-37.780772196999976, 175.22346050600004],
 [-37.789474094999946, 175.28109931100005],
 [-37.79174999999998, 175.2915200000001],
 [-37.82651999999996, 175.22323000000006],
 [-37.82651999999996, 175.22323000000006],
 [-37.7423912279

In [15]:
# inserting the coordinates into the Neighborhood_df by storing coordinates into coordinates_df
coordinates_df = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
Neighborhood_df['Latitude'] = coordinates_df['Latitude']
Neighborhood_df['Longitude'] = coordinates_df['Longitude']

In [16]:
# print and see the neighborhoods and the coordinates
print(Neighborhood_df.shape)
Neighborhood_df

(50, 3)


,Neighborhood,Latitude,Longitude
0,"Suburbs of Hamilton, New Zealand",-37.826520,175.223230
1,"Ashmore, New Zealand",-37.826520,175.223230
2,Beerescourt,-37.767203,175.262690
3,Callum Brae,-37.737305,175.270052
4,"Chartwell, Hamilton",-37.737680,175.274592
5,Chedworth Park,-37.753250,175.283870
6,Claudelands,-37.778600,175.287030
7,"Crawshaw, New Zealand",-42.287234,172.343433
8,Deanwell,-37.826033,175.276270
9,"Dinsdale, New Zealand",-37.795480,175.245810


In [17]:
# obtain Hamilton, New Zealand coordinates
address = 'Hamilton, New Zealand'

geolocator = Nominatim(user_agent="myGeocoder")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hamilton, New Zealand {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hamilton, New Zealand -37.7878809, 175.281788.


In [18]:
# create map of Hamilton, New Zealand using latitude and longitude values
map_Neighborhood = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(Neighborhood_df['Latitude'], Neighborhood_df['Longitude'], Neighborhood_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    #label = folium.Popup(label, parse_html=True)
    label = folium.Popup(str(label), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_Neighborhood)  
    
map_Neighborhood

In [20]:
# Get 100 venues with in 2000 meters 
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(Neighborhood_df['Latitude'], Neighborhood_df['Longitude'], Neighborhood_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [21]:
# convert the venues list into a new DataFrame
ven_df = pd.DataFrame(venues)

# define the column names
ven_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']


In [22]:
print(ven_df.shape)
ven_df.head()

(1093, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Suburbs of Hamilton, New Zealand",-37.826520,175.22323,POHLENKEAN,-37.826311,175.224842,Business Service
1,"Ashmore, New Zealand",-37.826520,175.22323,POHLENKEAN,-37.826311,175.224842,Business Service
2,Beerescourt,-37.767203,175.26269,Waterworld,-37.768226,175.252971,Water Park
3,Beerescourt,-37.767203,175.26269,The Helm Bar and Kitchen,-37.779572,175.274139,Bar
4,Beerescourt,-37.767203,175.26269,Sugar Bowl Cafe,-37.776321,175.258796,Café


In [23]:
# grouping by neighborhood to find out number of venues retuend by each neighborhood
ven_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Ashmore, New Zealand",1,1,1,1,1,1
Beerescourt,23,23,23,23,23,23
Callum Brae,31,31,31,31,31,31
"Chartwell, Hamilton",30,30,30,30,30,30
Chedworth Park,12,12,12,12,12,12
Claudelands,81,81,81,81,81,81
Deanwell,6,6,6,6,6,6
"Dinsdale, New Zealand",25,25,25,25,25,25
Enderley,15,15,15,15,15,15


In [25]:
print('Unique categories: {}.'.format(len(ven_df['VenueCategory'].unique())))

# print out the list of categories
ven_df['VenueCategory'].unique()[:50]

Unique categories: 90.


array(['Business Service', 'Water Park', 'Bar', 'Café',
       'Football Stadium', 'Supermarket', 'Pizza Place',
       'Arts & Crafts Store', 'Grocery Store', 'Thai Restaurant',
       'Hostel', 'Spa', 'Restaurant', 'Liquor Store', 'Moving Target',
       'Basketball Stadium', 'Fish & Chips Shop', 'Racetrack',
       'Burger Joint', 'Indian Restaurant', 'Coffee Shop', 'Steakhouse',
       'Shopping Mall', 'Fast Food Restaurant', 'Golf Course',
       'Multiplex', 'Park', 'Gym', 'Beer Store', 'Sandwich Place',
       'Bookstore', 'Middle Eastern Restaurant', 'Beer Bar',
       'Cocktail Bar', 'Hotel', 'Asian Restaurant', 'Mexican Restaurant',
       'Farmers Market', 'Performing Arts Venue', 'Japanese Restaurant',
       'Department Store', 'Wine Shop', 'Wine Bar', 'History Museum',
       'Ice Cream Shop', 'Brewery', 'Italian Restaurant', 'Movie Theater',
       'Cricket Ground', 'Plaza'], dtype=object)

In [26]:
# To make sure that the scope of project is correct here e check to see if the venue category contain "Café"
"Café" in ven_df['VenueCategory'].unique()

True

In [27]:
# Here we want ot create a new dataframe that stores neighborhood and their venue category
# First,creating new dataframe for venues category

Neighborhood_Ven_df = pd.get_dummies(ven_df[['VenueCategory']], prefix="", prefix_sep="")

Neighborhood_Ven_df.head(97)

,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,...,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Water Park,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [37]:
# Second,append neighborhood column back to Neighborhood_Ven_df
Neighborhood_Ven_df['Neighborhood'] = ven_df['Neighborhood'] 

print(Neighborhood_Ven_df.shape)
Neighborhood_Ven_df.head(97)

(1093, 92)


,Neighborhoods,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,...,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Water Park,Wine Bar,Wine Shop,Neighborhood
0,"Suburbs of Hamilton, New Zealand",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Suburbs of Hamilton, New Zealand"
1,"Ashmore, New Zealand",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Ashmore, New Zealand"
2,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Beerescourt
3,Beerescourt,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Beerescourt
4,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Beerescourt
5,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Beerescourt
6,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Beerescourt
7,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Beerescourt
8,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Beerescourt
9,Beerescourt,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Beerescourt


In [38]:
# Third, if you check neighborhood column is the last column, hence we move it to the first column

fixed_columns = [Neighborhood_Ven_df.columns[-1]] + list(Neighborhood_Ven_df.columns[:-1])
Neighborhood_Ven_df = Neighborhood_Ven_df[fixed_columns]

print(Neighborhood_Ven_df.shape)
Neighborhood_Ven_df.head(97)

(1093, 92)


,Neighborhood,Neighborhoods,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Court,Basketball Stadium,...,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Water Park,Wine Bar,Wine Shop
0,"Suburbs of Hamilton, New Zealand","Suburbs of Hamilton, New Zealand",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Ashmore, New Zealand","Ashmore, New Zealand",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Beerescourt,Beerescourt,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,Beerescourt,Beerescourt,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [39]:
# Here we group Neighborhood_Ven_df by neighberhood and put it in new dataframe
Neighborhoods_grouped = Neighborhood_Ven_df.groupby(["Neighborhood"]).mean().reset_index()

print(Neighborhoods_grouped.shape)
Neighborhoods_grouped


(49, 91)


,Neighborhood,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bar,Basketball Court,Basketball Stadium,Beer Bar,...,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Water Park,Wine Bar,Wine Shop
0,"Ashmore, New Zealand",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Beerescourt,0.000000,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,...,0.000000,0.000000,0.000000,0.086957,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000
2,Callum Brae,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Chartwell, Hamilton",0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Chedworth Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Claudelands,0.000000,0.000000,0.012346,0.012346,0.037037,0.049383,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.037037,0.012346,0.012346,0.000000,0.000000,0.012346,0.024691
6,Deanwell,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Dinsdale, New Zealand",0.080000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Enderley,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Fairfield, Hamilton",0.000000,0.018182,0.036364,0.000000,0.036364,0.018182,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.036364,0.018182,0.018182,0.000000,0.018182,0.000000,0.018182


In [40]:
# To go to final step, clustring, we need to create a new dataframe only for Cafe
Café_df = Neighborhoods_grouped[["Neighborhood","Café"]]
Café_df.head()

,Neighborhood,Café
0,"Ashmore, New Zealand",0.000000
1,Beerescourt,0.173913
2,Callum Brae,0.096774
3,"Chartwell, Hamilton",0.100000
4,Chedworth Park,0.083333


In [33]:
from sklearn.cluster import KMeans 

In [34]:
import sklearn.cluster.k_means_ as kmean
kmeans = kmean.KMeans()

In [41]:
# set number of clusters
Noclusters = 3

clustering_df = Café_df.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=Noclusters, random_state=0).fit(clustering_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 1], dtype=int32)

In [42]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Cluster_df = Café_df.copy()

# add clustering labels
Cluster_df["Cluster Labels"] = kmeans.labels_

# merge cafe_grouped with cafe_data to add latitude/longitude for each neighborhood
Cluster_df = Cluster_df.join(Neighborhood_df.set_index("Neighborhood"), on="Neighborhood")

print(Cluster_df.shape)
Cluster_df.head(50)

(49, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,"Ashmore, New Zealand",0.000000,0,-37.826520,175.223230
1,Beerescourt,0.173913,1,-37.767203,175.262690
2,Callum Brae,0.096774,0,-37.737305,175.270052
3,"Chartwell, Hamilton",0.100000,0,-37.737680,175.274592
4,Chedworth Park,0.083333,0,-37.753250,175.283870
5,Claudelands,0.160494,1,-37.778600,175.287030
6,Deanwell,0.166667,1,-37.826033,175.276270
7,"Dinsdale, New Zealand",0.080000,0,-37.795480,175.245810
8,Enderley,0.066667,0,-37.768193,175.296822
9,"Fairfield, Hamilton",0.181818,1,-37.769338,175.274991


In [46]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Noclusters)
ys = [i+x+(i*x)**2 for i in range(Noclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster_df['Latitude'], Cluster_df['Longitude'], Cluster_df['Neighborhood'], Cluster_df['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
Cluster_df.sort_values(["Cluster Labels"], inplace=True)
print(Cluster_df.shape)
Cluster_df.head(50)

(49, 5)


,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,"Ashmore, New Zealand",0.000000,0,-37.826520,175.223230
23,"Huntington, New Zealand",0.055556,0,-37.730424,175.282752
47,"Western Heights, Hamilton",0.090909,0,-37.788980,175.235430
29,"Naylor, New Zealand",0.000000,0,-37.826520,175.223230
30,"Peacocke, New Zealand",0.000000,0,-37.823000,175.313931
33,"Riverlea, New Zealand",0.071429,0,-37.807943,175.323663
34,Rotokauri,0.000000,0,-37.766372,175.190117
20,"Hamilton West, New Zealand",0.000000,0,-37.826520,175.223230
35,Rototuna,0.085714,0,-37.736173,175.269852
37,"Silverdale, Hamilton",0.071429,0,-37.796515,175.332405


In [49]:
#Cluster 0
Cluster_df.loc[Cluster_df['Cluster Labels'] == 0]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
0,"Ashmore, New Zealand",0.000000,0,-37.826520,175.223230
23,"Huntington, New Zealand",0.055556,0,-37.730424,175.282752
47,"Western Heights, Hamilton",0.090909,0,-37.788980,175.235430
29,"Naylor, New Zealand",0.000000,0,-37.826520,175.223230
30,"Peacocke, New Zealand",0.000000,0,-37.823000,175.313931
33,"Riverlea, New Zealand",0.071429,0,-37.807943,175.323663
34,Rotokauri,0.000000,0,-37.766372,175.190117
20,"Hamilton West, New Zealand",0.000000,0,-37.826520,175.223230
35,Rototuna,0.085714,0,-37.736173,175.269852
37,"Silverdale, Hamilton",0.071429,0,-37.796515,175.332405


In [56]:
#Cluster 1
Cluster_df.loc[Cluster_df['Cluster Labels'] == 1]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
1,Beerescourt,0.173913,1,-37.767203,175.262690
41,"St Petersburg, New Zealand",0.200000,1,-37.726836,175.239575
5,Claudelands,0.160494,1,-37.778600,175.287030
39,"St Andrews, Hamilton",0.160000,1,-37.752190,175.257990
38,Somerset Heights,0.196970,1,-37.787891,175.270899
6,Deanwell,0.166667,1,-37.826033,175.276270
9,"Fairfield, Hamilton",0.181818,1,-37.769338,175.274991
17,Hamilton Central,0.166667,1,-37.789474,175.281099
18,"Hamilton East, New Zealand",0.206349,1,-37.791750,175.291520
31,Pukete,0.178571,1,-37.735681,175.233064


In [51]:
#Cluster 2
Cluster_df.loc[Cluster_df['Cluster Labels'] == 2]

,Neighborhood,Café,Cluster Labels,Latitude,Longitude
27,"Melville, New Zealand",0.4,2,-37.813576,175.275873
46,"Thornton, Hamilton",0.5,2,-37.815716,175.275245


In [54]:
Cluster_df.groupby(["Cluster Labels"]).count()

,Neighborhood,Café,Latitude,Longitude
Cluster Labels,,,,
0,25,25,25,25
1,22,22,22,22
2,2,2,2,2


# Conclusion:

Based on the summery page above, most of the Cafe bars are located in Cluster in Melville and Thornton areas with highest number.

Compare to Custer 1 with moderate number, cluster 0 have very low number to no Cafe bar like as Temple View, Rotokauri and so on.

This means there are high competition in Melville and Thornton (Cluster 2) are another neighborhood in cluster 0 are starving of having Cafe bar. Also, the data illustrates that most of cafe bars are concentrated in central areas and not suburb area.
Therefore, the project recommends construction developers rely on these findings and invest of suburb areas more. Although those investors who are looking into more competition can also open cafe bars in neighborhood in cluster 1.
 



